In [1]:
import pandas as pd

# Define a Data Pipeline

In [20]:
class DataPipeline:
    @staticmethod
    def readFiles(file_path):
        """
        The function can load the file into one dataframe
        parameter: file_path - the target file's locations
        return: result_df
        """
        print(f"Processing: {file_path}")
        # Read file with all values as strings
        result_df = pd.read_csv(file_path, dtype=str, low_memory=False)
        print("✅ File loaded: Total ", len(result_df), " rows.")
        return result_df

    @staticmethod
    def filterData(df, filter_dict, use_regex=False):
        """
        filter function to apply multiple filters to a DataFrame.

        Parameters:
            df (pd.DataFrame): The DataFrame to filter.
            filter_dict (dict): Dictionary with:
                - key: column name
                - value: list of exact match values OR regex patterns
            use_regex (bool): If True, values in filter_dict will be interpreted as regex patterns.

        Returns:
            pd.DataFrame: Filtered DataFrame.
        """
        filtered_df = df.copy()
        for col, patterns in filter_dict.items():
            if col not in filtered_df.columns:
                print(f"⚠️ Column '{col}' not found in DataFrame. Skipping.")
                continue

            if use_regex:
                combined_pattern = "|".join(patterns)
                filtered_df = filtered_df[filtered_df[col].str.contains(combined_pattern, regex=True, na=False)]
            else:
                filtered_df = filtered_df[filtered_df[col].isin(patterns)]

        print(f"✅ Filter applied: {len(filtered_df)} rows retained.")
        return filtered_df


    @staticmethod
    def combineFiles(df_list):
        """
        Combine multiple DataFrames into one.
        
        Parameters:
            df_list (list of pd.DataFrame): List of DataFrames.
        
        Returns:
            pd.DataFrame: Combined DataFrame.
        """
        print(f"🔗 Combining {len(df_list)} dataframes...")
        combined_df = pd.concat(df_list, ignore_index=True)
        print(f"✅ Combined DataFrame has {len(combined_df)} rows.")
        return combined_df

    @staticmethod
    def downloadFiles(output_df, file_path, format="csv"):
        """
        Save a DataFrame to file.
        
        Parameters:
            output_df (pd.DataFrame): Data to save.
            file_path (str): Output path.
            format (str): File format. Default is "csv".
        
        Returns:
            None
        """
        if format == "csv":
            output_df.to_csv(file_path, index=False)
            print(f"✅ File saved as CSV: {file_path}")
        else:
            raise ValueError(f"Unsupported format: {format}")

# Filter Religious Organizations from IRS BMF Files
According to the description of National Taxonomy of Exempt Entities (NTEE) Code in the [eo-info file, P14](https://www.irs.gov/pub/irs-soi/eo-info.pdf), X is Religion-Related.

In [3]:
# List of IRS CSV files
DATA_ROOT = "Data/"
# List of IRS BMF data files
LIST_OF_IRS_CSV = [DATA_ROOT + i for i in ["eo1.csv", "eo2.csv", "eo3.csv", "eo4.csv"]]
# The 13 southern and southeastern states of interest
target_states = ['AL', 'AR', 'DC', 'FL', 'GA', 'KY', 'LA', 'MS',
                 'NC', 'SC', 'TN', 'VA', 'WV']

In [7]:
# Step 1: Read each file into a DataFrame
irs_dfs = [DataPipeline.readFiles(f) for f in LIST_OF_IRS_CSV]

# Step 2: Combine all IRS DataFrames into a single table
irs_combined = DataPipeline.combineFiles(irs_dfs)

# Step 3: Define regex filters to identify religious organizations
# - NTEE_CD starts with 'X'
# - CLASSIFICATION includes '07'
# - FILING_REQ_CD is '06' or '13'
religious_regex_filters = {
    "NTEE_CD": ["^X"]
}

# Apply regex filtering to find religious organizations
religious_df = DataPipeline.filterData(irs_combined, religious_regex_filters, use_regex=True)

# Step 4: Define state filter for the 13 target states
state_filters = {"STATE": target_states}

# Apply filtering to get only religious orgs in the target states
filtered_irs_df = DataPipeline.filterData(religious_df, state_filters, use_regex=False)

Processing: Data/eo1.csv
✅ File loaded: Total  269061  rows.
Processing: Data/eo2.csv
✅ File loaded: Total  696938  rows.
Processing: Data/eo3.csv
✅ File loaded: Total  916530  rows.
Processing: Data/eo4.csv
✅ File loaded: Total  4520  rows.
🔗 Combining 4 dataframes...
✅ Combined DataFrame has 1887049 rows.
✅ Filter applied: 188991 rows retained.
✅ Filter applied: 55576 rows retained.


In [22]:
DataPipeline.downloadFiles(filtered_irs_df, "Data/all_religion_groups_13states.csv")

✅ File saved as CSV: Data/all_religion_groups_13states.csv


# Load and Filter K–12 School Data

## ✅ Part A: Handle Public Schools

In [8]:
# Public school file (2023–24), already downloaded
public_school_path = DATA_ROOT + "ccd_sch_029_2324_w_1a_073124.csv"

In [9]:
# Load with DataPipeline
public_df = DataPipeline.readFiles(public_school_path)
# Use state filter
public_filtered = DataPipeline.filterData(public_df, {"ST": target_states}, use_regex=False)

Processing: Data/ccd_sch_029_2324_w_1a_073124.csv
✅ File loaded: Total  102274  rows.
✅ Filter applied: 22350 rows retained.


## ✅ Part B: Handle Private Schools

In [12]:
# Private school file (2021–22)
private_school_path = DATA_ROOT + "pss2122_pu.csv"
# Load with DataPipeline
private_df = DataPipeline.readFiles(private_school_path)
core_columns = ["PINST", "PADDRS", "PCITY", "PSTABB", "PZIP", "PPHONE", "LATITUDE22", "LONGITUDE22", "RELIG"]
private_df = private_df[core_columns]
# Use state filter
private_filtered = DataPipeline.filterData(private_df, {"PSTABB": target_states}, use_regex=False)

Processing: Data/pss2122_pu.csv
✅ File loaded: Total  22345  rows.
✅ Filter applied: 5703 rows retained.


## ✅ Part C: Combination

In [13]:
# Reload private data with exact fields as specified by the user
private_selected_strict = private_filtered[[
    "PINST", "PADDRS", "PCITY", "PSTABB", "PZIP", "PPHONE", "LATITUDE22", "LONGITUDE22", "RELIG"
]].rename(columns={
    "PINST": "School_Name",
    "PADDRS": "Address",
    "PCITY": "City",
    "PSTABB": "State",
    "PZIP": "Zip",
    "PPHONE": "Phone",
    "LATITUDE22": "Latitude",
    "LONGITUDE22": "Longitude",
    "RELIG": "Religious_Affiliation"
})
private_selected_strict["School_Type"] = "Private"
private_selected_strict["Website"] = None
private_selected_strict["District_Name"] = None

# Update public data to match private data structure
public_selected_strict = public_filtered[[
    "SCH_NAME", "MSTREET1", "LCITY", "ST", "LZIP", "PHONE", "WEBSITE"
]].rename(columns={
    "SCH_NAME": "School_Name",
    "MSTREET1": "Address",
    "LCITY": "City",
    "ST": "State",
    "LZIP": "Zip",
    "PHONE": "Phone",
    "WEBSITE": "Website"
})
public_selected_strict["Latitude"] = None
public_selected_strict["Longitude"] = None
public_selected_strict["Religious_Affiliation"] = None
public_selected_strict["School_Type"] = "Public"
public_selected_strict["District_Name"] = public_df["LEA_NAME"]

# Reorder columns to match
ordered_columns = [
    "School_Name", "Address", "City", "State", "Zip", "Phone",
    "Latitude", "Longitude", "Website", "Religious_Affiliation", "School_Type", "District_Name"
]
public_final = public_selected_strict[ordered_columns]
private_final = private_selected_strict[ordered_columns]

# Combine the datasets
combined_school_final = pd.concat([public_final, private_final], ignore_index=True)

In [15]:
len(combined_school_final)==len(public_filtered)+len(private_filtered)

True

In [18]:
combined_school_final.columns

Index(['School_Name', 'Address', 'City', 'State', 'Zip', 'Phone', 'Latitude',
       'Longitude', 'Website', 'Religious_Affiliation', 'School_Type',
       'District_Name'],
      dtype='object')

In [21]:
DataPipeline.downloadFiles(combined_school_final, "Data/all_k12_schools_13states.csv")

✅ File saved as CSV: Data/all_k12_schools_13states.csv
